In [92]:
import numpy as np
import osr_methods as osr
import line_search_methods as lsm
import ocr_methods as ocr
import numdifftools as nd
import matplotlib.pyplot as plt
from timeit import default_timer as timer

### Definição do Ponto Inicial

In [93]:
x = np.array([-0.5, 2])

### Escolha dos métodos de OSR e OCR

In [94]:
# Metodos OCR
# 1 - Penalidade
# 2 - Barreira
metodo_ocr = 2

# Metodos OSR
# 1 - Univariante
# 2 - Powell
# 3 - Stepest Descent
# 4 - Newton-Raphson
# 5 - Fletcher-Reeves
# 6 - BFGS
metodo_osr = 6

### Controle Numérico

In [95]:
# numero maximo de iteracoes na OSR
maxiter = 200

# tolerancia para convergencia do gradiente na OSR
tol_conv = 1E-5

# tolerancia para a busca unidirecional na OSR
tol_search = 1E-6

# delta alpha do passo constante na OSR
line_step = 1E-2

#epsilon da maquina
eps = 1E-8

#parametros ocr
if metodo_ocr == 1:
    #penalidade
    r = 1
    beta = 10
elif metodo_ocr == 2:
    #barreira
    r = 1
    beta = 0.1

#tolerancia OCR
tol = 1E-8

ctrl_num_osr = [maxiter, tol_conv, tol_search, line_step, eps]

### Definição de f

In [96]:
def f(x):
    return x[0]**2 + x[1]**2

def grad_f(x):
    return np.array([2*x[0], 2*x[1]])

def hess_f(x):
    hess = np.zeros((2,2), dtype=float)
    hess[0,:] = np.array([2,0])
    hess[1,:] = np.array([0,2])
    return hess

### Definição das restrições de igualdade

In [97]:
def hk1(x):
    return

def grad_hk1(x):
    return

def hess_hk1(x):
    return

def hk2(x):
    return

def grad_hk2(x):
    return

def hess_hk2(x):
    return

### Definição das restrições de desigualdade

In [98]:
def cl1(x):
    return 2*x[0] + x[1] - 2

def grad_cl1(x):
    return np.array([2,1])

def hess_cl1(x):
    return np.zeros((2,2), dtype=float)

def cl2(x):
    return -x[1] + 1

def grad_cl2(x):
    return np.array([0,-1])

def hess_cl2(x):
    return np.zeros((2,2), dtype=float)

### Listas de Restrições e suas derivadas

In [99]:
hk_list = [hk1, hk2]
grad_hk_list = [grad_hk1, grad_hk2]
hess_hk_list = [hess_hk1, hess_hk2]

cl_list = [cl1, cl2]
grad_cl_list = [grad_cl1, grad_cl2]
hess_cl_list = [hess_cl1, hess_cl2]

#para o metodo de penalidade
#controle de quais cls irao montar a phi
cl_mont = []
if metodo_ocr == 1:
    for cl in cl_list:
        if cl(x) > 0:
            cl_mont.append(1)
        else:
            cl_mont.append(0)
            
params = [f, grad_f, hess_f, hk_list, grad_hk_list, hess_hk_list, cl_list, grad_cl_list, hess_cl_list, cl_mont]


In [ ]:
if metodo_ocr == 1:
    parc = (1/2)*r*ocr.p_penal(x, params)
elif metodo_ocr == 2:
    parc = r*ocr.b_bar(x, params)
    
listP_OCR = []
listP_OCR.append(x)

passos_OCR = 0
while(parc > tol):
    passos_OCR = passos_OCR + 1
    if passos_OCR > 1:
        r = beta*r
        #atualiza cl_mont ....
    listP_OSR, passos_OSR, flag_conv_OSR, tempoExec_OSR = osr.osr_ctrl(x, params, r, ctrl_num_osr, metodo_ocr, metodo_osr)
    x = listP_OSR[-1]
    listP_OCR.append(x)
    if metodo_ocr == 1:
        parc = (1/2)*r*ocr.p_penal(x, hk_list, cl_list, cl_mont)
    elif metodo_ocr == 2:
        parc = r*ocr.b_bar(x, cl_list)
    